# Relativistic Raytracer

or, a guide in three days of procrastination and a 30% grade deduction

*brought to you by @chas-card, @owl10124, and @TheRealOrange*

## Motivation

I don't think the motivation for a relativistic raytracer needs to be explained. It's a relativistic raytracer.

In [ ]:
from array import array
from PIL import Image
import numpy as np
import math


np_type = np.float32
c = 8 #3e8

FARAWAY = 1.0e+39  # A large distance (i cant forsee how this will go wrong)

In [1]:
def norm(arr): return arr/np.sqrt(np.sum(np.square(arr),axis=0))
def lt_velo(lt, velo):
    if len(np.shape(velo))==1: velo=velo[:,np.newaxis]
    v4 = np.concatenate((np.array([c*np.ones(np.shape(velo)[1])]), velo), axis=0)
    vp4 = lt @ v4
    return vp4[1:]*c/vp4[0]

## Frame

The obvious important concept in SR: the transform between inertial reference frames.

Each frame has a velocity vector, giving rise to its own LT.


### Lorentz Transformation

Since we are dealing with a generalised LT which is capable of transforming reference frames which have relative velocities on more than one axis, the LT is in the form of a 4 by 4 matrix, shown below
$$
{\displaystyle \mathcal{L}(\mathbf {v} )={\begin{bmatrix}\gamma &-\gamma v_{x}/c&-\gamma v_{y}/c&-\gamma v_{z}/c\\-\gamma v_{x}/c&1+(\gamma -1){\dfrac {v_{x}^{2}}{v^{2}}}&(\gamma -1){\dfrac {v_{x}v_{y}}{v^{2}}}&(\gamma -1){\dfrac {v_{x}v_{z}}{v^{2}}}\\-\gamma v_{y}/c&(\gamma -1){\dfrac {v_{y}v_{x}}{v^{2}}}&1+(\gamma -1){\dfrac {v_{y}^{2}}{v^{2}}}&(\gamma -1){\dfrac {v_{y}v_{z}}{v^{2}}}\\-\gamma v_{z}/c&(\gamma -1){\dfrac {v_{z}v_{x}}{v^{2}}}&(\gamma -1){\dfrac {v_{z}v_{y}}{v^{2}}}&1+(\gamma -1){\dfrac {v_{z}^{2}}{v^{2}}}\end{bmatrix}}}
$$

Where $\gamma$ is the Lorentz factor, and $v_x$, $v_y$, $v_z$ are the 3 cartesian coordinate components of the velocity vector, and $v = \sqrt{{v_x}^2 + {v_y}^2 + {v_z}^2}$

### Defining Frames for Computation

We also define each frame relative to another frame. At the 'bottom', or the 'root' of the tree of references is a world frame. All the frames which are defined are either in the 'special' world frame or relative to it, either directly or recursively through another frame. This way, we can transform from any frame to any other frame by first transforming 'up' to the world frame, then transforming 'down' the chain of references to any other frame.

Furthermore, since LTs have the special property where multiplication with another LT also produces an LT, we can use this property to generate a single LT for transforming from one frame to another simply by multiplying all the LTs along the reference chain together

In [ ]:
# frame class to handle defining different reference frames with respect to other reference frames
# world frame is just (0, 0, 0, 0) position, use it as a "special" frame to transform between any frames
class Frame:
    def __init__(self, velocity, ref=None):
        self.velocity = np.array(velocity, dtype=np_type)
        self.ref = ref

    @property
    def lt(self):
        b=self.velocity/c
        b2 = np.sum(np.square(b))
        assert (b2 <= 1)

        g = 1 / (np.sqrt(1 - b2))

        lt_mat = np.eye(4, dtype=np_type)
        if b2==0: return lt_mat

        lt_mat[0, 0] = g
        lt_mat[0, 1:] = lt_mat[1:, 0] = -b * g
        lt_mat[1:, 1:] += (g - 1) * np.matmul(b[np.newaxis].T, b[np.newaxis]) / b2

        assert(abs(np.linalg.det(lt_mat)-1)<1e-3)
        return lt_mat

    @property
    def inv_lt(self):
        inv_lt_mat = self.lt
        inv_lt_mat[0, 1:] = -inv_lt_mat[0, 1:]
        inv_lt_mat[1:, 0] = -inv_lt_mat[1:, 0]
        return inv_lt_mat

    @property
    def to_world_lt(self):
        if self.ref is None:
            return self.inv_lt
        else:
            return np.matmul(self.ref.to_world_lt, self.inv_lt)

    @property
    def from_world_lt(self):
        if self.ref is None:
            return self.lt
        else:
            return np.matmul(self.lt, self.ref.from_world_lt)

    def compute_lt_to_frame(self, frame):
        return np.matmul(frame.from_world_lt, self.to_world_lt)

    def compute_lt_from_frame(self, frame):
        return np.matmul(self.from_world_lt, frame.to_world_lt)

    def __str__(self):
        return "Frame with velocity "+str(self.velocity)+" wrt "+str(self.ref)

## Camera

The screen / camera class does exactly what one expects.